shout out to YONDAM KIM(my teacher)

### 00. Data and Library Load

In [ ]:
# #if use TabPFN first time, add data(pip-package-icr) in your note book) and run this cell
# !pip download tabpfn --no-deps -d pip-packages
# !pip install tabpfn

In [ ]:
# if you already install tabpfn, off the internet and run this cell
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages

In [ ]:
# same as just before cell
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [ ]:
# basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import QuantileTransformer

# ML
from sklearn.ensemble import RandomForestClassifier  # Bagging
from xgboost.sklearn import XGBClassifier            # GBM
from sklearn.svm import SVC                          # SVM
from sklearn.utils.class_weight import compute_sample_weight

# NN
from sklearn.neural_network import MLPClassifier
from tabpfn import TabPFNClassifier

# KFold(CV), partial
from sklearn.model_selection import KFold, StratifiedKFold
from functools import partial
from sklearn.ensemble import VotingClassifier
from random import *
from sklearn.model_selection import train_test_split

# AutoML framework
import optuna

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred /= np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    
    return logloss

In [ ]:
def check_vif(df):
    vifs = [variance_inflation_factor(df, i) for i in range(df.shape[1])]
    vif_df = pd.DataFrame({"features":df.columns, "VIF" : vifs})
    vif_df = vif_df.sort_values(by="VIF", ascending=False)
    remove_col = vif_df.iloc[0, 0]
    top_vif = vif_df.iloc[0, 1]
    return vif_df, remove_col, top_vif

In [ ]:
# visualize experiment logs
def display_experiment_log(study):
    display(study.trials_dataframe())
    print("Best Score: %.4f" % study.best_value)
    print("Best params: ", study.best_trial.params)
    history = study.trials_dataframe()
    display(history[history.value == study.best_value])
    optuna.visualization.plot_optimization_history(study).show()
    optuna.visualization.plot_param_importances(study).show()

In [ ]:
# preprocessing
scaling = 'Quantile'                   # ['Standard','Robust','MinMax','Quantile'] or False
integerized = False
sampling_method = 'hybrid'             # ['under','SMOTE','ADASYN','ran_over','hybrid'] or False


#dimension_reduction
feature_extraction = False             # ['PCA','LDA'] or False
apply_vif = False
feature_selection = False              # ['permutation','rf_model'] or False

#ensmble
models = ['RF','XGB','SVM','TabPFN']
weights= [0.15, 0.35, 0, 0.5]         # equal number of models and weight or None
n_trials = 50
evaluation_metric = balance_logloss

#postprocessing
post = True

#CV
bag_num = 4

In [ ]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greek = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
print(train.shape, test.shape)

### 01. Preprocessing

In [1]:
lb = LabelEncoder()
train.EJ = lb.fit_transform(train.EJ)  # A->0, B->1
greek.Alpha = lb.fit_transform(greek.Alpha)
greek.Gamma = lb.fit_transform(greek.Gamma)
greek.Beta = lb.fit_transform(greek.Betaa)
greek.Delta = lb.fit_transform(greek.Delta)
greek = greek.drop(columns=["Id"])

train = train.drop(columns=["Id"])
train = pd.concat([train,greek], axis=1)

NameError: name 'LabelEncoder' is not defined

In [ ]:
if integerized:
    int_denominators = {
     'AB': 0.004273,
     'AF': 0.00242,
     'AH': 0.008709,
     'AM': 0.003097,
     'AR': 0.005244,
     'AX': 0.008859,
     'AY': 0.000609,
     'AZ': 0.006302,
     'BC': 0.007028,
     'BD ': 0.00799,
     'BN': 0.3531,
     'BP': 0.004239,
     'BQ': 0.002605,
     'BR': 0.006049,
     'BZ': 0.004267,
     'CB': 0.009191,
     'CC': 6.12e-06,
     'CD ': 0.007928,
     'CF': 0.003041,
     'CH': 0.000398,
     'CL': 0.006365,
     'CR': 7.5e-05,
     'CS': 0.003487,
     'CU': 0.005517,
     'CW ': 9.2e-05,
     'DA': 0.00388,
     'DE': 0.004435,
     'DF': 0.000351,
     'DH': 0.002733,
     'DI': 0.003765,
     'DL': 0.00212,
     'DN': 0.003412,
     'DU': 0.0013794,
     'DV': 0.00259,
     'DY': 0.004492,
     'EB': 0.007068,
     'EE': 0.004031,
     'EG': 0.006025,
     'EH': 0.006084,
     'EL': 0.000429,
     'EP': 0.009269,
     'EU': 0.005064,
     'FC': 0.005712,
     'FD ': 0.005937,
     'FE': 0.007486,
     'FI': 0.005513,
     'FR': 0.00058,
     'FS': 0.006773,
     'GB': 0.009302,
     'GE': 0.004417,
     'GF': 0.004374,
     'GH': 0.003721,
     'GI': 0.002572
    }
    for k, v in int_denominators.items():
        train[k] = np.round(train[k]/v,1)

In [ ]:
# fill missindata
train = train.fillna(train.median())

# check missing data
train[train.isnull().any(axis=1)]

In [ ]:
# seperate Class
X = train.drop(columns=["Class"])
y = train["Class"]

In [ ]:
# remove all features when VIF is high
if apply_vif:
    top_vif = 100

    while(top_vif > 10):
        vif_df, remove_col, top_vif = check_vif(X)
        print(remove_col, top_vif)
        if top_vif < 10:
            break
        X = X.drop(columns=remove_col)

In [ ]:
# Feature selection
if feature_selection == 'rf_model':
    rf = RandomForestClassifier()
    rf.fit(X, y)
    print("Train ACC : %.4f" % accuracy_score(y, rf.predict(X)))
    fi_df = pd.DataFrame({'feature':X.columns, 'importance':rf.feature_importances_})
    selected_cols = fi_df.sort_values(by="importance", ascending=False)[:20]["feature"].values
    X = X[selected_cols]

if feature_selection == 'permutation':
    rf = RandomForestClassifier(n_estimators=300, random_state=42)
    rf.fit(X, y)
    result = permutation_importance(rf, X, y,  
                                n_repeats=30, random_state=42, n_jobs=-1)
    # Feature label
    Feature = X.columns

    # sort by Feature importance(ascending), save index
    sorted_result = result.importances_mean.argsort()

    # # make DataFrame
    importances = pd.DataFrame(result.importances_mean[sorted_result], index=Feature[sorted_result]).sort_values(0, ascending=False)   
    selected_cols = list(importances[importances[0]>0][0].index)
    X = X[selected_cols]


In [ ]:
# class imbalance handling
## 1. undersampling
if sampling_method == 'under':
    c1 = X[y == 1]
    c0 = X[y == 0]
    print(c1.shape, c0.shape) # 108, 509 -> 108, 108
    c0 = c0.sample(n=c1.shape[0]) # 509 -> 108
    X = pd.concat([c0, c1])
    print(X.shape)
    
## 2. oversampling -> SMOTE
if sampling_method == 'SMOTE':
    smote = SMOTE(k_neighbors=5)
    # The fit_resample function automatically finds the minority class by y and fits it 1:1.
    X_resampled, y_resampled = smote.fit_resample(X, y)
    print(X_resampled.shape, y_resampled.shape)
    y = y_resampled
    X = X_resampled
    
    
if sampling_method == 'ran_over': 
    ros = RandomOverSampler(random_state=42)
    X, y = ros.fit_resample(X, y)
    
    
if sampling_method == 'ADASYN': 
    ros = ADASYN(random_state=42)
    X, y = ros.fit_resample(X, y)

    
    
# 3. hybrid approach
## class0 : 509 -> 300
## class1 : 108 -> 300
if sampling_method == 'hybrid':
    L = X.columns
    c1 = train[train.Class == 1]
    c0 = train[train.Class == 0]
    print(c1.shape, c0.shape)
    c0 = c0.sample(n=300) # 509 -> 300
    train = pd.concat([c0, c1])
    print(train.shape)
    
    X = train[L]
    y = train['Class']

    smote = SMOTE(k_neighbors=5)
    # The fit_resample function automatically finds the minority class by y and fits it 1:1.
    X, y = smote.fit_resample(X, y) # 300, 108 --> 300, 300
    print(X.shape, y.shape)

In [ ]:
# scaling
if scaling == 'Standard':
    scaler = StandardScaler()
    L = X.columns
    data_ = scaler.fit_transform(X)
    X = pd.DataFrame(data=data_, columns=X.columns)
    display(X)
    
if scaling == 'Robust':
    L = X.columns
    scaler = RobustScaler()
    data_ = scaler.fit_transform(X)
    X = pd.DataFrame(data=data_, columns=X.columns)
    display(X)
    
if scaling == 'MinMax':
    L = X.columns
    scaler =  MinMaxScaler()
    data_ = scaler.fit_transform(X)
    X = pd.DataFrame(data=data_, columns=X.columns)
    display(X)
    
if scaling == 'Quantile':
    L = X.columns
    scaler = QuantileTransformer(n_quantiles=50, random_state=42, output_distribution="normal")
    data_ = scaler.fit_transform(X)
    X = pd.DataFrame(data=data_, columns=X.columns)
    display(X)

In [ ]:
# feature extraction
if feature_extraction == 'PCA':    
    dim = PCA(n_components=0.90, random_state=42)
    data_ = dim.fit_transform(X)
    X = pd.DataFrame(data=data_, columns=[f"PC{i}" for i in range(1, data_.shape[1]+1)])
    X

if feature_extraction == 'LDA':
    dim = LinearDiscriminantAnalysis(n_components=6)
    data_ = dim.fit_transform(X, greek["Gamma"])
    X = pd.DataFrame(data=data_, columns=[f"PC{i}" for i in range(1, data_.shape[1]+1)])
    X

### 02.Hyper-parameter Tuning

In [ ]:
def rf_optimizer(trial, X, y, K):
    # define parameter to tune
    n_estimators = trial.suggest_categorical('n_estimators', [50, 100, 200])
    max_depth = trial.suggest_int('max_depth', 3, 8)
    max_features = trial.suggest_categorical('max_features', [0.6, 0.7, 0.8])
    
    
    # set model
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   max_features=max_features,
                                   criterion='log_loss',
#                                    class_weight='balanced'
                                  )
    
    # K-Fold Cross validation
    folds = StratifiedKFold(n_splits=K, shuffle=True)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)
    
    # return mean score of CV
    return np.mean(losses)

In [ ]:
if 'RF' in models:

    K = 5 # set K of K-Fold
    opt_func = partial(rf_optimizer, X=X, y=y, K=K)


    rf_study = optuna.create_study(direction="minimize") # determine minimize or maximize sth
    rf_study.optimize(opt_func, n_trials=n_trials)
    rf_best_params = rf_study.best_params
    best_rf = RandomForestClassifier(**rf_best_params)

    display_experiment_log(rf_study)

In [ ]:
def xgb_optimizer(trial, X, y, K):
    n_estimators = trial.suggest_categorical('n_estimators', [300, 500, 700])
    max_depth = trial.suggest_int('max_depth', 4, 10)
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8])
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 5e-2)
    reg_lambda = trial.suggest_categorical('reg_lambda', [0.5, 1, 2, 4])
#     sample_weight = trial.suggest_categorical('sample_weight', [sample_weights])

    
    model = XGBClassifier(n_estimators=n_estimators,
                          max_depth=max_depth,
                          colsample_bytree=colsample_bytree,
                          learning_rate=learning_rate,
                          reg_lambda=reg_lambda,
#                           objective="multi:softprob", # predict for greek["Alpha"]
                          eval_metric = evaluation_metric
#                          scale_pos_weight= 4.71
                            )  ## we set class imbalance by using sampling method.
    
    
    folds = StratifiedKFold(n_splits=K, shuffle=True)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)
    
    
    return np.mean(losses)

In [ ]:
if 'XGB' in models:

    K = 5
    opt_func = partial(xgb_optimizer, X=X, y=y, K=K)


    xgb_study = optuna.create_study(direction="minimize")
    xgb_study.optimize(opt_func, n_trials=n_trials)
    xgb_best_params = xgb_study.best_params
    best_xgb = XGBClassifier(**xgb_best_params)

    display_experiment_log(xgb_study)

In [ ]:
def svm_optimizer(trial, X, y, K):
    C = trial.suggest_int('C', 1, 30)
#     kernel = trial.suggest_categorical('kernel', ['rbf', 'linear'])
    gamma = trial.suggest_categorical('gamma', [0.1, 0.3, 0.5, 0.7, 1])
    

    model = SVC(C=C,
                kernel='rbf',
#                 class_weight='balanced', # if class imbalanced
                probability=True,
                #cache_size=1000,
                random_state=42,
                gamma = gamma,
                
               )
    
    folds = StratifiedKFold(n_splits=K, shuffle=True)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)
    
    
    return np.mean(losses)

In [ ]:
if 'SVM' in models:
    K = 5
    opt_func = partial(svm_optimizer, X=X, y=y, K=K)
    svm_study = optuna.create_study(direction="minimize")
    svm_study.optimize(opt_func, n_trials=n_trials)
    svm_best_params = svm_study.best_params
    best_svm = SVC(random_state=42, **svm_best_params, probability=True)

    display_experiment_log(svm_study)

In [ ]:
#TabPFN
tP = TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')

### 03. Test Prediction

In [ ]:
## preprocessing in same way
test.EJ = lb.fit_transform(test.EJ)  # A->0, B->1
test = test.drop(columns=["Id"])
X_test = test.fillna(train.median())

if scaling != False:
    X_test = scaler.transform(X_test[L])
    X_test = pd.DataFrame(data=X_test, columns=L)

if feature_extraction != False:
    data_ = dim.transform(X_test)
    X_test = pd.DataFrame(data=data_, columns=[f"PC{i}" for i in range(1, data_.shape[1]+1)])

X_test = X_test[X.columns]
X_test

### 04. Fitting and Evaluation(w/OOF)

In [ ]:
voting_clf = VotingClassifier(estimators=[
                ["rf", best_rf],
                ["xgb", best_xgb],
                ["svc", best_svm],
                ["TabPFN", tP]
    ],
    voting="soft"
    , weights=weights
)

In [ ]:
def oof_preds(best_model):
    avg_tlosses = []
    avg_vlosses = []
    
    for bag in range(bag_num):

        print(f'########################## bag: {bag} ##########################')

        folds = StratifiedKFold(n_splits=K, random_state=42 * bag, shuffle=True)
        train_scores = []
        val_scores = []
        models[bag] = []

        for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
            X_train = X.iloc[train_idx, :]
            y_train = y.iloc[train_idx]
            X_val = X.iloc[val_idx, :]
            y_val = y.iloc[val_idx]

            best_model.fit(X_train, y_train)
            pred_train = best_model.predict_proba(X_train)
            pred_val= best_model.predict_proba(X_val) 
            train_score = evaluation_metric(y_train, pred_train)
            val_score = evaluation_metric(y_val, pred_val)
            train_scores.append(train_score)
            val_scores.append(val_score)

        avg_tloss = np.mean(train_scores)
        avg_vloss = np.mean(val_scores)
        print(f"Bags: {bag}, Train Score: {round(avg_tloss, 3)}")
        print(f"Bags: {bag}, Test Score: {round(avg_vloss, 3)}")
        
        avg_tlosses.append(avg_tloss)
        avg_vlosses.append(avg_vloss)
        
    fi_tloss = np.mean(avg_tlosses)
    fi_vloss = np.mean(avg_vlosses)


    return fi_tloss, fi_vloss



In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)


print("Random Forest")
rf_tloss, rf_vloss = oof_preds(best_model=best_rf)
print("--- Prediction Score ---")
print("Train Score : %.4f" % rf_tloss)
print("Test Score : %.4f" % rf_vloss)

print("\nXGBoost")
xgb_tloss, xgb_vloss = oof_preds(best_model=best_xgb)
print("--- Prediction Score ---")
print("Train Score : %.4f" % xgb_tloss)
print("Test Score : %.4f" % xgb_vloss)

print("\nSupport Vector Machine")
svm_tloss, svm_vloss = oof_preds(best_model=best_svm)
print("--- Prediction Score ---")
print("Train Score : %.4f" % svm_tloss)
print("Test Score : %.4f" % svm_vloss)

print("\nTabPFN")
tP_tloss, tP_vloss = oof_preds(best_model=tP)
print("--- Prediction Score ---")
print("Train Score : %.4f" % tP_tloss)
print("Test Score : %.4f" % tP_vloss)


print("\nEnsemble model")
ens_tloss, ens_vloss = oof_preds(best_model=voting_clf)
print("--- Prediction Score ---")
print("Train Score : %.4f" % ens_tloss)
print("Test Score : %.4f" % ens_vloss)

### 05. Submission

In [ ]:
voting_clf.fit(X, y)
preds = voting_clf.predict_proba(X_test)

In [ ]:
#post processing
if post:
    p0 = preds[:, 0]
    p0[p0 > 0.85] = 1
    p0[p0 < 0.15] = 0
else:
    p0 = preds[:, 0]

In [ ]:
submission = pd.read_csv('../input/icr-identify-age-related-conditions/sample_submission.csv')
submission

In [ ]:
submission['class_0'] = p0
submission['class_1'] = 1-p0
submission

In [ ]:
submission.to_csv("submission.csv", index=False)